In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls
%cd drive/MyDrive/Datasets_new/
!ls

drive  sample_data
/content/drive/MyDrive/Datasets_new
data-prep.csv


In [4]:
ds = pd.read_csv("data-prep.csv")
ds

,Unnamed: 0,referee,date,venue_name,venue_city,elapsed,league_id,league_name,league_country,season,...,away_mean_6_passes_accurate,away_mean_12_passes_accurate,away_mean_passes_%,away_mean_6_passes_%,away_mean_12_passes_%,away_team_score,away_team_rang,away_mean_team_score,away_mean_6_team_score,away_mean_12_team_score
0,0,E. Abbattista,2021-01-06 11:30:00+00:00,Sardegna Arena,Cagliari,90.0,135.0,Serie A,Italy,2020.0,...,NaN,NaN,343.500000,NaN,NaN,3.0,0.0,0.000000,NaN,NaN
1,1,J. Sacchi,2021-01-06 14:00:00+00:00,Gewiss Stadium,Bergamo,90.0,135.0,Serie A,Italy,2020.0,...,NaN,NaN,430.000000,NaN,NaN,-3.0,-6.0,-3.000000,NaN,NaN
2,2,M. Bacher,2021-01-06 17:30:00+00:00,flyeralarm-Arena,Würzburg,90.0,79.0,2. Bundesliga,Germany,2020.0,...,NaN,NaN,229.000000,NaN,NaN,-3.0,-3.0,-3.000000,NaN,NaN
3,3,Carlos Del Cerro,2021-01-06 20:00:00+00:00,San Mamés Barria,Bilbao,90.0,140.0,La Liga,Spain,2020.0,...,NaN,NaN,725.000000,NaN,NaN,3.0,3.0,3.000000,NaN,NaN
4,4,"Romain Delpech, France",2021-01-08 19:00:00+00:00,Stade Auguste-Bonal,Montbéliard,90.0,62.0,Ligue 2,France,2020.0,...,NaN,NaN,368.500000,NaN,NaN,1.0,4.0,2.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,9142,"Guillaume Paradis, France",2023-01-31 19:45:00+00:00,Parc des Sports,Annecy,90.0,62.0,Ligue 2,France,2022.0,...,371.833333,377.916667,343.974359,81.000000,81.500000,3.0,6.0,0.076923,0.666667,0.333333
9143,9143,"Faouzi Benchabane, France",2023-01-31 19:45:00+00:00,Stade Francis Le Basser,Laval,90.0,62.0,Ligue 2,France,2022.0,...,354.833333,361.416667,322.250000,80.666667,80.166667,-3.0,-46.0,-0.605263,0.000000,-1.166667
9144,9144,"Olivier Thual, France",2023-01-31 19:45:00+00:00,Stade Auguste-Bonal,Montbéliard,90.0,62.0,Ligue 2,France,2022.0,...,375.666667,350.000000,308.730769,82.500000,80.583333,-3.0,2.0,0.025641,-1.000000,-0.166667
9145,9145,"William Toulliou, France",2023-01-31 19:45:00+00:00,Stade René Gaillard,Niort,90.0,62.0,Ligue 2,France,2022.0,...,378.666667,368.750000,352.065789,81.333333,81.000000,3.0,20.0,0.263158,-1.333333,0.166667


In [5]:
# Очистка и обработка данных
ds['draw'] = ds['draw'].replace({True: 1, False: 0})
ds['away_team_winner'] = ds['away_team_winner'].replace({True: 1, False: 0})
ds['home_team_winner'] = ds['home_team_winner'].replace({True: 1, False: 0})

all_names = []
for away_name in ds['away_team_name'].tolist():
  if away_name not in all_names:
    all_names.append(away_name)

for home_name in ds['home_team_name'].tolist():
  if home_name not in all_names:
    all_names.append(home_name)
from sklearn.preprocessing import LabelEncoder
mapping = {}
for i in range(len(all_names)):
  mapping[all_names[i]] = i
ds['home_team_name'] = ds['home_team_name'].replace(mapping)
ds['away_team_name'] = ds['away_team_name'].replace(mapping)
le = LabelEncoder()
 
le.fit(ds['season_round'])
ds['season_round'] = le.transform(ds['season_round'])
 
le.fit(ds['league_country'])
ds['league_country'] = le.transform(ds['league_country'])
 
le.fit(ds['league_name'])
ds['league_name'] = le.transform(ds['league_name'])
 
le.fit(ds['venue_city'])
ds['venue_city'] = le.transform(ds['venue_city'])
 
le.fit(ds['venue_name'])
ds['venue_name'] = le.transform(ds['venue_name'])
 
le.fit(ds['date'])
ds['date'] = le.transform(ds['date'])
 
le.fit(ds['referee'])
ds['referee'] = le.transform(ds['referee'])
clean_df = ds.dropna()


Предсказываем ничью:

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

y = clean_df['draw']
x = clean_df.drop(['draw'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

y_pred_dtc = dtc.predict(X_test)

acc_dtc = accuracy_score(y_test, y_pred_dtc)
print("Decision Tree Classifier Accuracy:", acc_dtc)

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

acc_rfc = accuracy_score(y_test, y_pred_rfc)
print("Random Forest Classifier Accuracy:", acc_rfc)

svc = SVC(random_state=42)
svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)

acc_svc = accuracy_score(y_test, y_pred_svc)
print("Support Vector Classifier Accuracy:", acc_svc)

mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

y_pred_mlp = mlp.predict(X_test)

acc_mlp = accuracy_score(y_test, y_pred_mlp)
print("Multi-layer Perceptron Classifier Accuracy:", acc_mlp)

Decision Tree Classifier Accuracy: 1.0
Random Forest Classifier Accuracy: 0.9881035689293212
Support Vector Classifier Accuracy: 0.7207837648705389
Multi-layer Perceptron Classifier Accuracy: 0.6256123163051085


Предсказываем победу home_team:

In [9]:

y = clean_df['home_team_winner']
x = clean_df.drop(['home_team_winner'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

y_pred_dtc = dtc.predict(X_test)

acc_dtc = accuracy_score(y_test, y_pred_dtc)
print("Decision Tree Classifier Accuracy:", acc_dtc)

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

acc_rfc = accuracy_score(y_test, y_pred_rfc)
print("Random Forest Classifier Accuracy:", acc_rfc)

svc = SVC(random_state=42)
svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)

acc_svc = accuracy_score(y_test, y_pred_svc)
print("Support Vector Classifier Accuracy:", acc_svc)

mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

y_pred_mlp = mlp.predict(X_test)

acc_mlp = accuracy_score(y_test, y_pred_mlp)
print("Multi-layer Perceptron Classifier Accuracy:", acc_mlp)

Decision Tree Classifier Accuracy: 1.0
Random Forest Classifier Accuracy: 0.9909027291812457
Support Vector Classifier Accuracy: 0.5955213435969209
Multi-layer Perceptron Classifier Accuracy: 0.5325402379286214


Предсказываем победу away_team:

In [10]:

y = clean_df['away_team_winner']
x = clean_df.drop(['away_team_winner'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

y_pred_dtc = dtc.predict(X_test)

acc_dtc = accuracy_score(y_test, y_pred_dtc)
print("Decision Tree Classifier Accuracy:", acc_dtc)

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

acc_rfc = accuracy_score(y_test, y_pred_rfc)
print("Random Forest Classifier Accuracy:", acc_rfc)

svc = SVC(random_state=42)
svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)

acc_svc = accuracy_score(y_test, y_pred_svc)
print("Support Vector Classifier Accuracy:", acc_svc)

mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

y_pred_mlp = mlp.predict(X_test)

acc_mlp = accuracy_score(y_test, y_pred_mlp)
print("Multi-layer Perceptron Classifier Accuracy:", acc_mlp)

Decision Tree Classifier Accuracy: 1.0
Random Forest Classifier Accuracy: 0.9762071378586424
Support Vector Classifier Accuracy: 0.7095871238628412
Multi-layer Perceptron Classifier Accuracy: 0.7046885934219734


Предсказываем счет домашней команды:

In [11]:

y = clean_df['home_team_score']
x = clean_df.drop(['home_team_score'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

y_pred_dtc = dtc.predict(X_test)

acc_dtc = accuracy_score(y_test, y_pred_dtc)
print("Decision Tree Classifier Accuracy:", acc_dtc)

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

acc_rfc = accuracy_score(y_test, y_pred_rfc)
print("Random Forest Classifier Accuracy:", acc_rfc)

svc = SVC(random_state=42)
svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)

acc_svc = accuracy_score(y_test, y_pred_svc)
print("Support Vector Classifier Accuracy:", acc_svc)

mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

y_pred_mlp = mlp.predict(X_test)

acc_mlp = accuracy_score(y_test, y_pred_mlp)
print("Multi-layer Perceptron Classifier Accuracy:", acc_mlp)

Decision Tree Classifier Accuracy: 0.7200839748075577
Random Forest Classifier Accuracy: 0.7347795661301609
Support Vector Classifier Accuracy: 0.4359692092372288
Multi-layer Perceptron Classifier Accuracy: 0.36529041287613717


Предсказываем счет команды гостей:

In [12]:


y = clean_df['away_team_score']
x = clean_df.drop(['away_team_score'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(X_train, y_train)

y_pred_dtc = dtc.predict(X_test)

acc_dtc = accuracy_score(y_test, y_pred_dtc)
print("Decision Tree Classifier Accuracy:", acc_dtc)

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

y_pred_rfc = rfc.predict(X_test)

acc_rfc = accuracy_score(y_test, y_pred_rfc)
print("Random Forest Classifier Accuracy:", acc_rfc)

svc = SVC(random_state=42)
svc.fit(X_train, y_train)

y_pred_svc = svc.predict(X_test)

acc_svc = accuracy_score(y_test, y_pred_svc)
print("Support Vector Classifier Accuracy:", acc_svc)

mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)

y_pred_mlp = mlp.predict(X_test)

acc_mlp = accuracy_score(y_test, y_pred_mlp)
print("Multi-layer Perceptron Classifier Accuracy:", acc_mlp)

Decision Tree Classifier Accuracy: 0.6724982505248426
Random Forest Classifier Accuracy: 0.6843946815955213
Support Vector Classifier Accuracy: 0.443666899930021
Multi-layer Perceptron Classifier Accuracy: 0.4632610216934919
